# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122737e+02     1.592957e+00
 * time: 0.6891539096832275
     1     1.105512e+01     9.618065e-01
 * time: 3.210170030593872
     2    -1.178277e+01     1.035717e+00
 * time: 3.8008508682250977
     3    -3.381059e+01     8.631083e-01
 * time: 4.72448992729187
     4    -4.732473e+01     6.752086e-01
 * time: 5.646566867828369
     5    -5.659649e+01     2.424961e-01
 * time: 6.538017988204956
     6    -5.947681e+01     1.964552e-01
 * time: 7.166640043258667
     7    -6.071827e+01     7.084547e-02
 * time: 7.792325973510742
     8    -6.127306e+01     4.212211e-02
 * time: 8.41960597038269
     9    -6.157320e+01     4.078909e-02
 * time: 9.073256015777588
    10    -6.176176e+01     3.107483e-02
 * time: 9.688848972320557
    11    -6.193052e+01     2.297163e-02
 * time: 10.302114009857178
    12    -6.203556e+01     1.679735e-02
 * time: 10.933093070983887
    13    -6.208374e+01     1.486867e-02
 * time: 11.558602094650269
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671061
    AtomicLocal         -18.8557696
    AtomicNonlocal      14.8522673
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485367 
    Xc                  -19.3336818

    total               -62.261666452663
